In [1]:

import requests
import pandas as pd
import hopsworks
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import json
import re
import os
import sys
from dotenv import load_dotenv
import warnings
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
import util

warnings.filterwarnings("ignore")

In [ ]:
load_dotenv()
proj = hopsworks.login()
fs = proj.get_feature_store() 

In [ ]:
el_prices_fg = fs.get_feature_group(
    name='el_prices',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
power_fg = fs.get_feature_group(
    name='power',
    version=1,
)

In [5]:
date = datetime.date.today()
yesterday = date - timedelta(days=1)
el_price_df = util.get_el_price(yesterday)
el_price_df

,price,date
0,0.164415,2024-12-28


In [ ]:
el_prices_fg.insert(el_price_df, write_options={"wait_for_job": True})

In [7]:
latitude = 59.329323
longitude = 18.068581

weather_data = util.get_daily_weather(1, 0, latitude, longitude)

weather_data.head()

Coordinates 59.32889938354492°N 18.072357177734375°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                       date  temperature_2m_max  sunshine_duration  \
0 2024-12-28 00:00:00+00:00            6.163334                0.4   

   precipitation_sum  wind_speed_10m_max  wind_direction_10m_dominant  
0          16.199999          233.302689                          0.0  


,date,temperature_2m_max,sunshine_duration,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
0,2024-12-28 00:00:00+00:00,6.163334,0.4,16.199999,233.302689,0.0


In [ ]:
weather_fg.insert(weather_data, write_options={"wait_for_job": True})

In [8]:
load_dotenv()
ELECTRICITY_API_TOKEN = os.getenv("ELECTRICTY_MAP_API_KEY")
power_data = util.fetch_data_for_yesterday(ELECTRICITY_API_TOKEN)
yesterday_formated = yesterday.strftime("%d.%m.%Y")
power_data.insert(0, "date", yesterday_formated)
print(power_data)

Data fetched successfully:
      hydro_mw   nuclear_mw    other_mw      wind_mw
0  7520.291667  5781.708333  578.541667  7376.416667
         date     hydro_mw   nuclear_mw    other_mw      wind_mw
0  28.12.2024  7520.291667  5781.708333  578.541667  7376.416667


In [ ]:
power_fg.insert(power_data, write_options={"wait_for_job": True})

Coordinates 59.32889938354492°N 18.072357177734375°E
Elevation 24.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
                       date  temperature_2m_max  sunshine_duration  \
0 2024-12-24 23:00:00+00:00            9.217500           0.000000   
1 2024-12-25 23:00:00+00:00            9.117499           0.000000   
2 2024-12-26 23:00:00+00:00            7.967500        1492.621582   
3 2024-12-27 23:00:00+00:00            6.717500           0.000000   
4 2024-12-28 23:00:00+00:00            7.917500           0.000000   
5 2024-12-29 23:00:00+00:00            7.667500       13238.261719   
6 2024-12-30 23:00:00+00:00            0.917500       13910.627930   

   precipitation_sum  wind_speed_10m_max  wind_direction_10m_dominant  
0                0.2           20.160000                   240.726639  
1                0.0           24.119999                   264.867737  
2                3.4           16.919998                   277.060303  
3       